<a href="https://colab.research.google.com/github/Karthikeyan-Ravichandran22/PysparkRegressions/blob/master/Pyspark_Liner%20Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Pratical_session").getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=fcb9844c176cb498186b5fcde1348f12b04442b2a4927f7280b706ae5c9767e6
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [5]:
from pyspark.ml.regression import LinearRegression

In [4]:
!wget https://raw.githubusercontent.com/krishnaik06/PysparkRegressions/master/Ecommerce_Customers.csv

--2023-02-13 19:06:52--  https://raw.githubusercontent.com/krishnaik06/PysparkRegressions/master/Ecommerce_Customers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66193 (65K) [text/plain]
Saving to: ‘Ecommerce_Customers.csv’

Ecommerce_Customers 100%[===================>]  64.64K  --.-KB/s    in 0.007s  

2023-02-13 19:06:52 (8.74 MB/s) - ‘Ecommerce_Customers.csv’ saved [66193/66193]



In [6]:
dataset=spark.read.csv("/content/Ecommerce_Customers.csv",inferSchema=True,header=True)

In [8]:
dataset.show()

+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|               Email|             Address|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|       34.49726773|12.65565115|    39.57766802|         4.082620633|         587.951054|
|   hduke@hotmail.com|4547 Archer Commo...|       31.92627203|11.10946073|    37.26895887|         2.664034182|        392.2049334|
|    pallen@yahoo.com|24645 Valerie Uni...|       33.00091476|11.33027806|    37.11059744|         4.104543202|        487.5475049|
|riverarebecca@gma...|1414 David Throug...|       34.30555663|13.71751367|    36.72128268|         3.120178783|         581.852344|
|mstephens@davidso...|14023 Rodriguez P...|       33.33067252|12.79518855|  

In [9]:
dataset.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [18]:
featureAssembler = VectorAssembler(inputCols=["Avg Session Length","Time on App","Time on Website","Length of Membership"],outputCol="Indepent Feature")

In [19]:
output=featureAssembler.transform(dataset)

In [21]:
output.select("Indepent Feature").show()

+--------------------+
|    Indepent Feature|
+--------------------+
|[34.49726773,12.6...|
|[31.92627203,11.1...|
|[33.00091476,11.3...|
|[34.30555663,13.7...|
|[33.33067252,12.7...|
|[33.87103788,12.0...|
|[32.0215955,11.36...|
|[32.73914294,12.3...|
|[33.9877729,13.38...|
|[31.93654862,11.8...|
|[33.99257277,13.3...|
|[33.87936082,11.5...|
|[29.53242897,10.9...|
|[33.19033404,12.9...|
|[32.38797585,13.1...|
|[30.73772037,12.6...|
|[32.1253869,11.73...|
|[32.33889932,12.0...|
|[32.18781205,14.7...|
|[32.61785606,13.9...|
+--------------------+
only showing top 20 rows



In [23]:
Final_model = output.select("Indepent Feature","Yearly Amount Spent")

In [24]:
train_data, test_data = Final_model.randomSplit([0.75,0.25])

In [25]:
Regressor= LinearRegression(featuresCol="Indepent Feature",labelCol="Yearly Amount Spent")

In [27]:
Regressor=Regressor.fit(train_data)

In [28]:
Regressor.coefficients

DenseVector([25.4222, 38.4082, 0.603, 61.7869])

In [29]:
Regressor.intercept

-1044.8815364341979

In [32]:
prediction=Regressor.evaluate(test_data)

In [35]:
prediction.predictions.show()

+--------------------+-------------------+------------------+
|    Indepent Feature|Yearly Amount Spent|        prediction|
+--------------------+-------------------+------------------+
|[30.39318454,11.8...|        319.9288698|331.76147215592005|
|[31.38958548,10.9...|        410.0696111|410.19577229538186|
|[31.42522688,13.2...|        530.7667187| 534.7184119590181|
|[31.6739155,12.32...|        475.7250679|  502.298439145389|
|[31.7207699,11.75...|        538.7749335| 546.5889371015414|
|[31.82797906,12.4...|        440.0027475|449.23453612948015|
|[31.82934646,11.2...|         385.152338| 384.2724740222336|
|[31.90962683,11.3...|        563.4460357| 552.4191955499834|
|[31.95630056,12.8...|        547.1259317| 564.9747321737607|
|[32.01230077,12.1...|        492.9450531| 489.4367586884887|
|[32.04781463,12.4...|        497.3895578| 480.4074345122083|
|[32.08838063,11.9...|        512.1658664| 518.2121287203101|
|[32.09610899,10.8...|        375.3984554| 375.4010094905086|
|[32.122